In [3]:
# install lib for translate
!pip install googletrans==3.1.0a0

# Import all lib need for crawl and translation
from bs4 import BeautifulSoup
import requests
import googletrans
from googletrans import Translator

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 6.0 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16352 sha256=aae63e480aea57f045aad5beffcf330e4198203c880a9bd932564741e4217fe8
  Stored in directory: /root/.cache/pip/wheels/50/5d/3c/8477d0af4ca2b8b1308812c09f1930863caeebc762fe265a95
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling cha

# Algorithm 1:
Code crawl data from ieltstrainingonline.com with BeautifulSoup

In [4]:
# Crawl data from ieltstrainingonline.com with BeautifulSoup
def crawlDataIeltstrainingonline():
  with open('data.txt', 'w') as f:
    # crawl all data of cambridge book 10 - 18, reading 4 test
    for i in range(10, 19):
      for j in range(1,5):
        url = "https://ieltstrainingonline.com/practice-cambridge-ielts-"+str(i)+"-reading-test-0"+str(j)+"-with-answer/"
        html_content = requests.get(url).text
        soup = BeautifulSoup(html_content, "html.parser")
        texts = soup.find_all('p')
        for text in texts:
          # get paragraph with len > 10
          if len((text.get_text()).strip()) > 10:
            # get paragraph not include special char
            if '……………' not in text.get_text() and (text.get_text())[0].isdigit() != True and len((text.get_text()).strip()) > 20:
              # write data to file
              f.write((text.get_text()).strip())
              f.write('\n')

In [7]:
# crawl all data from ieltstrainingonline.com
crawlDataIeltstrainingonline()

# Algorithm 2:
Code get all URL data pages of web site, prepare for crawl data with URLs

In [8]:
# crawl all link data pages mini ielts.com, hrw.org
def crawlAllURL(page_start, page_end, url_web, url_write, name_file):
  HEADERS = {'User-Agent': 'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148'}
  with open('url.txt', 'w') as f:
    for i in range(page_start, page_end+1):
      # get url and all link tag a in page of this url
      url = url_web+str(i)
      html_content = requests.get(url, headers=HEADERS).text

      soup = BeautifulSoup(html_content, "html.parser")
      texts = soup.find_all('a')
      for text in texts:
        if text:
          if text.has_attr('href'):
            # write all url satisfies with conditions
            if 'reading/' in text['href'] and 'view-solution' not in text['href'] and 'https://www.hrw.org/' not in text['href'] and 'https://www.hrw.org/' not in text['href'] and 'dispatches' not in text['href'] and 'video-photos' not in text['href'] and 'interview' not in text['href'] and 'news-release' not in text['href']:
              f.write((url_write+text['href']).strip())
              f.write('\n')
  # remove duplicate
  data = open("/content/url.txt")
  all_url = data.readlines()
  urls = [*set(all_url)]

  # write all url to file
  with open("/content/"+name_file, 'w') as f:
    for line in urls:
      f.write(line.strip())
      f.write('\n')

In [10]:
# crawl reading test URL in  mini ielts.com
crawlAllURL(1, 55, "https://mini-ielts.com/reading?page=", "https://mini-ielts.com", "url_mini_test.txt")

# crawl news data URL in hrw.org
crawlAllURL(1, 726, "https://www.hrw.org/languages?language=vi&page=", "https://www.hrw.org", "url_hrw.txt")

# Algorithm 3:
Code crawl all data mini-ielts.com with url

In [11]:
def crawlDataMiniIelts(urls):
  with open('data.txt', 'w') as f:
    for i in urls:
      if i.strip():
        url = i.strip()
        html_content = requests.get(url).text
        # get all data of page
        soup = BeautifulSoup(html_content, "html.parser")
        # get main data in tag <div class="reading-text">
        div = soup.find("div", class_="reading-text")

        # write data to file
        f.write((div.get_text()).strip())
        f.write('\n')

In [14]:
# read all URL
data = open("/content/url_mini_test.txt")
urls = data.readlines()

# crawl all data with URL
crawlDataMiniIelts(urls)

# Algorithm 4:
Code crawl all data hrw.org with url

In [15]:
# crawl all data hrw.org with url
def crawlDataHrw(urls):
  HEADERS = {'User-Agent': 'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148'}
  with open('data.txt', 'w') as f:
    for i in urls:
      url = i.strip()
      html_content = requests.get(url, headers=HEADERS).text
      # get all data of page
      soup = BeautifulSoup(html_content, "html.parser")

      # get main data in tag <div class ="WordSection2">
      divs = soup.find("div", class_="WordSection2")
      if divs:
        # get data in tag p, i
        p = divs.find_all('p')
        li = divs.find_all('li')
        if p:
          for pt in p:
            if pt.get_text().strip() != '' and len(pt.get_text().strip()) > 10:
              # write data to file
              f.write(pt.get_text().strip())
              f.write('\n')
        if li:
          for lit in li:
            if lit.get_text().strip() != '' and len(lit.get_text().strip()) > 10:
              # write data to file
              f.write(lit.get_text().strip())
              f.write('\n')
      else:
        # get main data in tag <div class ="article-body">
        div1s = soup.find("div", class_="article-body")
        if div1s:
          # get data in tag p
          p = div1s.find_all('p')
          if p:
            for pt in p:
              if pt.get_text().strip() != '' and len(pt.get_text().strip()) > 10:
                # write data to file
                f.write(pt.get_text().strip())
                f.write('\n')

In [ ]:
# read all URL
data = open("/content/url_hrw.txt")
urls = data.readlines()

# crawl all data with URL
crawlDataHrw(urls)

# Algorithm 5:
Crawl all data CNN with urls (manual handling)

In [16]:
# Crawl all data CNN with urls
def crawlDataCNN(url):
  with open('data_cnn.txt', 'w') as f:
    for i in urls:
      url = i.strip()
      html_content = requests.get(url).text
      # get all data of page
      soup = BeautifulSoup(html_content, "html.parser")
      # get main data in tag <div class="article__content-container"
      div = soup.find("div", class_="article__content-container")
      if div:
        # get data in tag p
        texts = div.find_all('p')
        for text in texts:
          if len((text.get_text()).strip()) > 100:
            # write data to file
            f.write((text.get_text()).strip())
            f.write('\n')

In [ ]:
# read all URL
data = open("/content/url_cnn.txt")
urls = data.readlines()

# crawl all data with URL
crawlDataCNN(urls)

# Divide data
Divide data with one sentence in one line

In [2]:
# read data in file
data = open("/content/data.txt") #change name data file for divide
data = data.readlines()

lines = []
lines_wrong = []
for line in data:
  if(line != '\n' and line != ''):
    # divide data with '. '
    if '. ' in line:
      # except special cases
      if 'Mr. ' not in line and 'Mrs. ' not in line and 'Dr. ' not in line and 'p.m. ' not in line and 'a.m. ' not in line:
        ln = line.split('. ')
        for ln_c in ln:
          # remove empty data and data too short
          if ln_c != '\n' and ln_c != '' and len(ln_c) > 10:
            lines.append((ln_c.replace("\n", "")).strip())
      else:
        lines_wrong.append(line.replace("\n", "").strip())
    else:
      if line.strip():
        ln_c1 = line.replace("\n", "")
        lines.append(ln_c1.strip())

# write to file with same name
with open('data.txt', 'w') as f:
  for line in lines:
    f.write(line)
    f.write('\n')

# Algorithm 6:
Code translate all data English to Vietnamese use googletrans

In [17]:
# Translate all data English to Vietnamese use googletrans
def translateData(data):
  ftranslate = Translator()

  result = []
  # remove ‘\n’ in data
  for en in contents:
    result.append(en.replace('\n',''))
  # call Translator to translate data to Vietnamese
  result_vie = ftranslate.translate(result, dest='vi')

  vie = []
  for x in result_vie:
    # get result translate
    vie.append(x.text)
  # write all Vietnamese to vi.txt file
  with open('vi.txt', 'w') as f:
      for line in vie:
          f.write(line)
          f.write('\n')

In [ ]:
# read English data for translate, run with small data in a time (divide data file crawled)
data_en = open("/content/data_en.txt")
contents = data_en.readlines()

# translate all data
translateData(contents)